In [1]:
from tqdm.notebook import tqdm
import pandas as pd
pd.set_option("display.max_colwidth", None)
import json
from sentence_transformers import SentenceTransformer, util

In [2]:
df = pd.read_csv("../data/valid_df.csv", keep_default_na=False)

In [3]:
df.head()

,Unnamed: 0,arg_id,key_point_id,label,argument,topic,stance,key_point
0,0,arg_4_121,kp_4_5,1,"A real education is about giving students the tools to learn, think, and express themselves; dictating to them what to wear sends a strong message that we don't trust them to think on their own.",We should abandon the use of school uniform,1,School uniform is harming the student's self expression
1,1,arg_4_122,kp_4_5,1,children express themselves through the clothes they wear and should be able to do this at school,We should abandon the use of school uniform,1,School uniform is harming the student's self expression
2,2,arg_4_123,kp_4_5,1,"children should be able to dress as they wish, within reason, at school rather than being restricted from expressing themselves through their clothes.",We should abandon the use of school uniform,1,School uniform is harming the student's self expression
3,3,arg_4_124,kp_4_5,1,children should be allowed to express themselves,We should abandon the use of school uniform,1,School uniform is harming the student's self expression
4,4,arg_4_126,kp_4_5,1,freedom of expression extends to the right to choose one's own clothing and a school uniform prevents this choice.,We should abandon the use of school uniform,1,School uniform is harming the student's self expression


In [4]:
model = SentenceTransformer('paraphrase-distilroberta-base-v1')

In [5]:
def get_paraphrase_score(argument, keypoint):
    paraphrases = util.paraphrase_mining(model, [argument, keypoint])
    score, i, j = paraphrases[0]
    return score

In [6]:
def match_argument_with_keypoints(result, kp_dict, arg_dict):
    for arg_id, arg in arg_dict.items():
        result[arg_id] = {}
        for kp_id, kp in kp_dict.items():
            result[arg_id][kp_id] = get_paraphrase_score(arg, kp)
    return result

In [7]:
valid_df = df

In [8]:
argument_keypoints = {}
for topic in tqdm(valid_df.topic.unique()):
    for stance in [-1, 1]:
        topic_keypoint_ids = valid_df[(valid_df.topic==topic) & (valid_df.stance==stance)]['key_point_id'].tolist()
        topic_keypoints = valid_df[(valid_df.topic==topic) & (valid_df.stance==stance)]['key_point'].tolist()
        topic_kp_dict = dict(zip(topic_keypoint_ids, topic_keypoints))
        
        topic_argument_ids = valid_df[(valid_df.topic==topic) & (valid_df.stance==stance)]['arg_id'].tolist()
        topic_arguments = valid_df[(valid_df.topic==topic) & (valid_df.stance==stance)]['argument'].tolist()
        topic_arg_dict= dict(zip(topic_argument_ids, topic_arguments))
        # match 
        argument_keypoints = match_argument_with_keypoints(argument_keypoints, topic_kp_dict, topic_arg_dict)

In [9]:
len(argument_keypoints)

932

In [10]:
json.dump(argument_keypoints, open("paraphrase_detection_all_valid_predictions.json", "w",encoding='utf-8'))

In [12]:
! python3 ../src-py/track_1_kp_matching.py ../data paraphrase_detection_all_valid_predictions.json our_valid

mAP strict= 0.7114848640492888 ; mAP relaxed = 0.878784591933353
